## CALCULATE AVERAGE THRESHOLD IN STELLAR LATEST USED

In [22]:
import re
import pandas as pd
import sys

log_path = 'src/ledger_logs.txt'
pattern = re.compile(r"needed=(\d+)")
needed = []

with open(log_path) as f:
    for line in f:
        m = pattern.search(line)
        if m:
            needed.append(int(m.group(1)))

df_needed = pd.DataFrame({'needed': needed})
avg_needed = df_needed['needed'].mean()

display(df_needed)

FileNotFoundError: [Errno 2] No such file or directory: 'src/ledger_logs.txt'

In [ ]:
print(f"Average 'needed' (minimum_quorum) across all checks: {avg_needed:.2f}")

In [ ]:
import json
import pandas as pd

json_path = 'src/quorumset_05_06_2025.json'
with open(json_path) as f:
    raw = json.load(f)

def flatten_list(inner):
    result = []
    for v in inner.get('validators', []):
        result.append(v)
    for sub in inner.get('innerQuorumSets', []):
        result.extend(flatten_list(sub))
    return result

rows = []
for entry in raw['nodes']:
    pk = entry['publicKey']
    qs = entry['quorumSet']
    thr = qs['threshold']
    all_ids = set(qs.get('validators', []))
    for sub in qs.get('innerQuorumSets', []):
        all_ids.update(flatten_list(sub))
    peer_count = len(all_ids)
    threshold_percent = thr / peer_count if peer_count > 0 else None
    rows.append({
        'node': pk,
        'peer_count': peer_count,
        'threshold': thr,
        'threshold_percent': threshold_percent
    })

df_peers = pd.DataFrame(rows)
avg_peers = df_peers['peer_count'].mean()
avg_percent = df_peers['threshold_percent'].dropna().mean()

display(df_peers)


In [ ]:
print(f"Average number of unique peers per node: {avg_peers:.2f}")
print(f"Average threshold as a fraction of peers: {avg_percent:.2%}")

In [ ]:
import json
import pandas as pd

json_path = 'src/quorumset_05_06_2025.json'
with open(json_path) as f:
    raw = json.load(f)

def flatten_list(inner):
    # collect *validators* itself, but for slice-count we don't flatten deeper
    return inner.get('validators', [])

rows = []
for entry in raw['nodes']:
    pk = entry['publicKey']
    qs = entry['quorumSet']
    thr = qs['threshold']
    top_validators    = len(qs.get('validators', []))
    top_slices        = 1                           # the top-level slice itself counts as 1
    inner_slices      = len(qs.get('innerQuorumSets', []))
    total_slices      = top_slices + inner_slices
    # what % of slices does JSON-threshold represent?
    threshold_pct     = thr / total_slices if total_slices>0 else None

    rows.append({
        'node'             : pk,
        'total_slices'     : total_slices,
        'json_threshold'   : thr,
        'threshold_percent': threshold_pct
    })

df = pd.DataFrame(rows)

print(f"avg. slices/node = {df['total_slices'].mean():.1f}")
print(f"avg. JSON threshold = {df['json_threshold'].mean():.1f}")
print(f"avg. % of slices = {100*df['threshold_percent'].mean():.1f}%")
